# Projeto 2: Teste AB

## Integrantes: LUCA NOTO, BRUNO PORTO E FELIPE GIORGI

** Durante as aulas de Ciência dos Dados do segundo semestre, fomos propostos a aplicar um teste A/B em uma das páginas WEB desenvolvidas durante o curso de engenharia do insper. **

### O que é o teste A/B?

**Este teste é muito utilizado por empresas para conhecerem melhor seu público e identificar qual abordagem funciona melhor para seu site. O teste A/B consiste em dividir o tráfego de uma determidada página em duas versões diferentes, uma é a atual e a outra é uma espécie de desafiante. Em ambas as versões os usuários são propostos a realizar uma tarefa semelhante e a partir de algumas inferências estatísticas é possível decidir qual versão é a mais eficiente.  **

**Neste projeto, as duas páginas a serem testadas são os portifólios de projetos de dois grupos de alunos do INSPER, que foram desenvolvidos para mesma finalidade, porém o site A tem um menu com botões grandes e centralizados e o site B, um menu na parte superior e do tipo scroll. A tarefa a ser realizada é encontrar um projeto específico dentro dos portifólios. Para cada usuário que realiza o teste é registrado o tempo que foi necessário para realizar tal tarefa. Ao final da coleta de dados, buscamos concluir qual a melhor página baseando-se na análise das medias populacionais do tempo da tarefa.**


In [17]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
import math
from scipy import stats
import numpy as np

# Técnica Inferencial

A técnica inferencial a ser usada neste projeto será teste de hipóteses.

In [18]:
dados = pd.read_csv("dados_preparados.csv", sep=",")

In [19]:
dados.head()

,Unnamed: 0,delta_t,user_id,versao
0,0,59.713,285365760,B
1,1,41.444,3789181448,A
2,2,18.755,528519713,A
3,3,24.113,2830537766,A
4,4,26.782,2409261628,B


## Obtendo medidas resumo das Amostras

In [20]:
na=0
tempoa=0

nb=0
tempob=0


for i in range(len(dados)):
    versao = dados.iloc[i]['versao']
    if versao == "A":
        na+=1
        tempoa+=dados.iloc[i]['delta_t']
    elif versao == "B":
        nb+=1
        tempob+=dados.iloc[i]['delta_t']
else: 
    pass
    
media_a = tempoa/na
media_b = tempob/nb


print("N˚ A:",na)
print("N˚ B:",nb)
print()
print("Tempo total A:",tempoa)
print("Tempo total B:",tempob)
print()
print("Média A:",media_a)
print("Média B:",media_b)

N˚ A: 29
N˚ B: 28

Tempo total A: 1726.261
Tempo total B: 1597.781

Média A: 59.5262413793
Média B: 57.0636071429


In [21]:
dpa=0
dpb=0

for i in range(len(dados)):
    versao = dados.iloc[i]['versao']
    if versao == "A":
        dpa+=((dados.iloc[i]['delta_t']-media_a)**2)
    elif versao == "B":
        dpb+=((dados.iloc[i]['delta_t']-media_b)**2)
else: 
    pass

dpa = math.sqrt((dpa/na))
dpb = math.sqrt((dpb/nb))

print("Desvio Padrão de A:",dpa)
print("Desvio Padrão de B:",dpb)

Desvio Padrão de A: 47.7644612610927
Desvio Padrão de B: 52.74187919639469


## Teste de hipótese para ver se as variâncias são iguais com nível de significância de 10%

Antes de analisar as medias populacionais do tempo necessário para a realização da tarefa é necessário descobrir se os desvios padrões das amostras podem ser considerados iguais ou não. Dependendo do resultado deste teste, a estatística usada na análise das medias pode ser diferente.

### Hipóteses Iniciais:

**Ho:** sigmaA = sigmaB

**Ha:** sigmaA != sigmaB

In [22]:
grau_de_liberdadeA = na - 1
grau_de_liberdadeB = nb - 1

nivel_significancia = 0.1
print("Nível de Significância:",nivel_significancia*100,"%")
print()

porcentagem_direita = nivel_significancia/2
porcentagem_esquerda= 1 - porcentagem_direita

valor_min_f = stats.f.ppf(porcentagem_direita,grau_de_liberdadeA,grau_de_liberdadeB)
print("F Crítico Mínimo:",valor_min_f)
print()

valor_max_f = stats.f.ppf(porcentagem_esquerda,grau_de_liberdadeA,grau_de_liberdadeB)
print("F Crítico Máximo:",valor_max_f)
print()

Nível de Significância: 10.0 %

F Crítico Mínimo: 0.529261805012

F Crítico Máximo: 1.89752281059



In [23]:
f_obs=(dpb**2)/(dpa**2)
print('F observado:',f_obs)
print()

if f_obs < valor_max_f and f_obs > valor_min_f:
    print('Para a distribuição F(',grau_de_liberdadeA,',',grau_de_liberdadeB,') ao nível de significância de 10% o intervalo de confiança está contido em {',valor_min_f,',',valor_max_f,'}.')
    print()
    

F observado: 1.219274334292867

Para a distribuição F( 28 , 27 ) ao nível de significância de 10% o intervalo de confiança está contido em { 0.529261805012 , 1.89752281059 }.



**O f observado encontra-se dentro do intervalo, portanto a Hipóteses Ho não pode ser rejeitada com uma confiança de 10%. Ou seja, pode-se afirmar que os desvios-padrões das amostras são iguais. Isso nos permite fazer a análise das medias populacionais estimando apenas um valor para o desvio-padrão populacional, deixando nossa análise mais precisa.**

## Análise das medias populacionais.
    
Nesta etapa do projeto, testaremos se há, de fato uma página mais eficiente que a outra. Para isso faremos um teste de hipóteses onde a hipótese inicial é que as medias populacionais são iguais.

### Hipóteses:

**Ho:** miA = miB

**Ha:** miA != miB

In [24]:
grau_liberdade_t = na + nb - 2
print("Grau de liberdade:",grau_liberdade_t)
print()

nivel_significancia = 0.1
print("Nivel de significância:",nivel_significancia*100,"%")
print()

valor_min_t =  stats.t.ppf(nivel_significancia,grau_liberdade_t)
valor_max_t = -stats.t.ppf(nivel_significancia,grau_liberdade_t)
print("T Crítico Mínimo:",valor_min_t)
print("T Crítico Máximo:",valor_max_t)
print()

s_quadrado = (((na-1) * (dpa**2)) +((nb-1) * (dpb**2)))/(na+nb-2)

t_obs = (media_a-media_b)/((s_quadrado/na)+(s_quadrado/nb))
print("T observado:",t_obs)

Grau de liberdade: 55

Nivel de significância: 10.0 %

T Crítico Mínimo: -1.2971342998
T Crítico Máximo: 1.2971342998

T observado: 0.0138826128444


In [25]:
if t_obs > valor_min_t and f_obs < valor_max_t:
    print('O t observado encontra-se dentro do intervalo, portanto a Hipóteses Ho não pode ser rejeitada com uma confiança de 10%')

O t observado encontra-se dentro do intervalo, portanto a Hipóteses Ho não pode ser rejeitada com uma confiança de 10%


## Conclusão

** Após o teste de hipótese feito com as medias populacionais das páginas A e B, pode-se afirmar que não há diferença o bastante entre as interfaces testadas que tornem uma delas de fato mais eficiente que a outra. Caso quisessemos melhorar a usabilidade de nosso portifólio, seria necessário fazer novas alterações na página 'desafiante' e realizar o teste A/B novamente, em busca de uma interface que superasse de fato a antiga. **